## Collect data

In [15]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains

In [20]:
options = webdriver.ChromeOptions()
#options.add_argument('--headless=new')
browser = webdriver.Chrome(options=options)
urls = [
    "https://web3sec.notion.site/ba459372dc434341b99ec92a932f98dc?v=7fceca7b3da74aa8a99b49c44a2a3916",
    "https://web3sec.notion.site/c582b99cd7a84be48d972ca2126a2a1f?v=4671590619bd4b2ab16a15256e4fbba1",

]
htmls = []
raw  = []

for url in urls:
    try:
        # Wait for the element with the ID of wrapper
        browser.get(url)
        wrapper = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'notion-table-view-row'))
        )
        last_height = browser.execute_script("return document.body.scrollHeight")
        while True:
            browser.execute_script("window.scrollTo(100, document.body.scrollHeight);")
            sleep(2)  # Adjust this delay as needed

            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        wrapper = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'notion-table-view-row'))
        )
        raw.append(browser.page_source)
        htmls.append(BeautifulSoup(wrapper.text, 'html.parser'))
    except TimeoutException:
        print("Element did not show up")



C:\Users\Abhi\AppData\Local\Temp\ipykernel_8168\3990664347.py:32: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  htmls.append(BeautifulSoup(wrapper.text, 'html.parser'))


In [51]:
#BeautifulSoup(raw[0], 'html.parser').find_all('div', class_='notion-table-view')

[<div class="notion-table-view" style="position: relative; float: left; min-width: 100%; user-select: none; font-variant-numeric: lining-nums tabular-nums; padding-bottom: 180px; padding-left: 96px; padding-right: 96px;"><div class="notion-selectable notion-collection_view_page-block" data-block-id="ba459372-dc43-4341-b99e-c92a932f98dc" style="position: relative;"><div class="notion-table-view-header-row" style="display: flex; position: absolute; background: rgb(25, 25, 25); z-index: 87; height: 33px; color: rgba(255, 255, 255, 0.443); border-top: 1px solid rgb(47, 47, 47); box-shadow: rgb(25, 25, 25) -3px 0px 0px, rgb(47, 47, 47) 0px 1px 0px; min-width: calc(100% - 192px); left: 0px; right: 0px;"><div style="display: inline-flex; margin: 0px;"><div style="display: flex; background: rgb(25, 25, 25); position: sticky; left: 0px; z-index: 83;"></div><div style="display: flex; position: relative;"><div class="notion-table-view-header-cell" style="display: flex; flex-shrink: 0; overflow: h

In [21]:
tables = []

for r in raw:
    tables.append(BeautifulSoup(r, 'html.parser').find('div', class_='notion-table-view'))

    

In [22]:
ts = []

for table in tables:
    t = []
    rows = table.findChildren('div', class_='notion-table-view-row')
    for row in rows:
        cells = row.findChildren('div', class_='notion-table-view-cell')
        r = []
        for cell in cells:
            if cell.findChildren('a', href=True):
                href = cell.findChildren('a', href=True)[0]['href']
                if href[:5] == 'https':
                    r.append(href)
                else:
                    r.append(f"https://web3sec.notion.site{href}")    
                continue
            r.append(cell.text)
        t.append(r)
    ts.append(t)
  
ts[1]

[['Miscalculation',
  '20230601',
  'Cellframenet',
  'https://web3sec.notion.site/Miscalculation-during-liquidity-migration-d4be7cafe1e541a1985cbcce5c0f13fb?pvs=25',
  '$76k',
  'https://github.com/SunWeb3Sec/DeFiHackLabs/blob/main/src/test/Cellframe_exp.sol'],
 ['Price Manipulation',
  '20230531',
  'ERC20TokenBank',
  'https://web3sec.notion.site/Price-manipulation-3f5479aa208e4cfd9ecdc6f7c20c217e?pvs=25',
  '$111k',
  'https://github.com/SunWeb3Sec/DeFiHackLabs/blob/main/src/test/ERC20TokenBank_exp.sol'],
 ['Price Manipulation',
  '20230529',
  'Jimbo',
  'https://web3sec.notion.site/Protocol-Specific-Price-Manipulation-4e3c752119df448791a656ebf445789e?pvs=25',
  '$8M',
  'https://github.com/SunWeb3Sec/DeFiHackLabs/blob/main/src/test/Jimbo_exp.sol'],
 ['Slippage',
  '20230529',
  'BabyDogeCoin',
  'https://web3sec.notion.site/Lack-Slippage-Protection-1828cba4d0cb43dda50b6c1e7f14ea57?pvs=25',
  '$135k',
  'https://github.com/SunWeb3Sec/DeFiHackLabs/blob/main/src/test/BabyDogeCoin_ex

In [80]:
tables[0].findChildren('div', class_='notion-table-view-row')[0].findChildren('div', class_='notion-table-view-cell')[].findChildren('a', href=True)

[]

In [23]:
len(tables[0].findChildren('div', class_='notion-table-view-row'))

101

In [24]:
len(ts[0])

101

In [25]:
options.add_argument('--headless=new')
browser = webdriver.Chrome(options=options)
for t in ts:
    for row in t:
        code_link = row[3]
        print("Fetching code link: ", code_link)
        try:
            # Wait for the element with the ID of wrapper
            browser.get(code_link)
            wrapper = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'notion-code-block'))
            )
            code_block = BeautifulSoup(browser.page_source, 'html.parser').find('div', class_='notion-code-block')
            row.append(code_block.text)
        except TimeoutException:
            print("Code Element did not show up")
            row.append("No Code")
            
        try:
            # Wait for the element with the ID of wrapper
            browser.get(code_link)
            wrapper = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'notion-text-block'))
            )
            text = BeautifulSoup(browser.page_source, 'html.parser').find('div', class_='notion-text-block')
            text = ' '.join([t.text for t in text])
            row.append(text)
        except TimeoutException:
            print("Text Element did not show up")
            row.append("")

       
        

Fetching code link:  https://web3sec.notion.site/Insufficient-validation-a593dd953d9f4bb692e1359acf052dc4?pvs=25
Fetching code link:  https://web3sec.notion.site/Incorrect-calculation-3628bf9f5a084c73bba1bb1ee4946c62?pvs=25
Fetching code link:  https://web3sec.notion.site/Insufficient-validation-04e0c2b7105c44e98dd2576e18f68835?pvs=25
Fetching code link:  https://web3sec.notion.site/Incorrect-access-control-fb2894e0ae804b9fb9c35e76f55db4d0?pvs=25
Fetching code link:  https://web3sec.notion.site/Incorrect-access-control-3b0318547ef74808b56c9f1ed435f55d?pvs=25
Fetching code link:  https://web3sec.notion.site/Insufficient-validation-6f07ce0ceae944f2baef2219b555b18b?pvs=25
Fetching code link:  https://web3sec.notion.site/Insufficient-validation-f1b3a9f5d3914151b928a53b36741488?pvs=25
Code Element did not show up
Fetching code link:  https://web3sec.notion.site/Incorrect-price-calculation-via-balanceOf-4dc3898d9dc549c79178174ba38a4bcf?pvs=25
Fetching code link:  https://web3sec.notion.site/

In [12]:
options = webdriver.ChromeOptions()
#options.add_argument('--headless=new')
browser = webdriver.Chrome(options=options)
code_link = ts[0][25][3]
print(code_link)
try:
    # Wait for the element with the ID of wrapper
    browser.get(code_link)
    wrapper = WebDriverWait(browser, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, 'notion-code-block'))
    )
    code_block = BeautifulSoup(browser.page_source, 'html.parser').find('div', class_='notion-code-block')
    print(code_block.text)
except TimeoutException:
    print("Code Element did not show up")
    print("No Code")
    
try:
    # Wait for the element with the ID of wrapper
    browser.get(code_link)
    wrapper = WebDriverWait(browser, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, 'notion-text-block'))
    )
    text = BeautifulSoup(browser.page_source, 'html.parser').find_all('div', class_='notion-text-block')
    text = ' '.join([t.text for t in text])
    print(text)
except TimeoutException:
    print("Text Element did not show up")
    print("")
    

https://web3sec.notion.site/Incorrect-reward-calculation-7f373b865943410ab75a405327461193?pvs=25
SolidityCopyfunction 0x6811e3b9() public nonPayable {
require(_isAirAddr.code.size);
    v0, v1 = _isAirAddr.balanceOf(msg.sender).gas(msg.gas);
require(v0); // checks call status, propagates error data on error
require(RETURNDATASIZE() >= 32);
require(v1 > 0, 'Amount can not be Zero');
if (owner_d[msg.sender] <= 0) {
        owner_d[msg.sender] = stor_6;
}
    v2 = _SafeDiv(stor_8, block.timestamp - owner_d[msg.sender]);
require(v2 > 0, 'The collection time was not reached');
    v3 = v4 = 0;
if (block.timestamp > stor_7) { //vulnerable point
if (v2 > 0) {
            v5 = 0x3182(stor_b, v1);
            v3 = v6 = _SafeDiv(0xf4240, v5);
}
} else if (v2 > 0) { //vulnerable point
        v7 = 0x3182(stor_b, v1);
        v8 = 0x3182(v2, v7);
        v3 = v9 = _SafeDiv(0xf4240, v8);
}
require(_isAirAddr.code.size);
    v10, v11 = _isAirAddr.transfer(msg.sender, v3).gas(msg.gas);
require(v10); 

In [102]:
ts[0][-1]

['DAOFlashloansPricemanipulationskim',
 '20220524',
 'HackDao',
 'https://web3sec.notion.site/Incorrect-logic-in-_transfer-function-of-token-contract-74f17f2adf084fd5a493e03aadd761ac?pvs=25',
 '',
 'https://github.com/SunWeb3Sec/DeFiHackLabs#20220524-hackdao---skim-token-balance',
 '',
 'SolidityCopyfunction _transfer(\naddress sender,\naddress recipient,\nuint256 amount\n    ) internal override virtual {\nrequire(sender != address(0), "ERC20: transfer from the zero address");\nrequire(recipient != address(0), "ERC20: transfer to the zero address");\n_beforeTokenTransfer(sender, recipient, amount);\nuint256 senderBalance = _balances[sender];\nrequire(senderBalance >= amount, "ERC20: transfer amount exceeds balance");\n        unchecked {\n            _balances[sender] = senderBalance - amount;\n}\nuint256 actualAmount = amount;\nif(parentAddress[recipient] == address(0) && recipient != uniswapV2Pair){\nif(sender == uniswapV2Pair){\n                parentAddress[recipient] = _defaultAdd

In [26]:
with open('data_verbose.csv', 'w', encoding="utf-8") as f:
    for t in ts:
        for row in t:
            f.write(f"{','.join(row)}\n")